In [229]:
import numpy as np

In [230]:
def conv_sqrt_n(n, limit):  # return (limit) convergents of the square root of n A_i
    ai = [0, 1]
    a = int(np.floor(np.sqrt(n)))
    t = float(np.sqrt(n) - a)

    for i in range(2, limit + 2):
        ai.append(a * ai[i-1] + ai[i-2])  # use recurrence A_i = a*A_i-1 + A_i-2 for convergents A
        a = int(np.floor(1 / t))
        t = float((1 / t) - a)
    return ai[2:]  # exclude initiating values

In [231]:
print(conv_sqrt_n(17873, 10))
print(conv_sqrt_n(556, 5))

[133, 134, 401, 1738, 3877, 13369, 17246, 47861, 65107, 178075]
[23, 24, 47, 118, 165]


Helpful functions: Legendre symbol, sieve prime test, set B of prime factors p with with n quadratic residue mod p and checking if a number is B-smooth, returning its prime powers from the B set.

In [232]:
def legendre(n, p): # legendre symbol (n/p)
    if n % p == 0:
        return 0
    if n > p:
        n = n % p
    if n ** (int((p - 1) / 2)) % p == 1:
        return 1
    else:
        return -1


def smsieve(n):  # prime check with classic sieving
    if n in [1, 2]:
        return 1
    for i in range(2, int(np.floor(np.sqrt(n))) + 1):
        if n % i == 0:
            return 0
    return 1

def Bset(k, n):  # construct the set of the first k primes p that have (n/p) = +1 together with -1
    B = [-1]
    p = 2
    while k:
        while legendre(n, p) != 1 or smsieve(p) != 1:
            p += 1
        B.append(p)
        p += 1
        k = k - 1
    return B

def Bfactors(n, B):  # for number n and set of primes B with -1 returns the factorisation and powers of n if n is
                     # B-smooth and 0 if n is not B-smooth
    if n < 0:
        basepowers = [1]
        n = int(n * -1)
    else:
        basepowers = [0]
    B = B[1:]
    for prime in B:
        prime_power = 0
        while n % prime == 0:
            n = int(n / prime)
            prime_power += 1
        basepowers.append(prime_power)
    if n == 1:
        return basepowers
    else:
        return 0

In [233]:
def least_res(a, n):
    a = a % n
    if a >= np.floor(n / 2):
        a = a - n
    return a

In [234]:
print(legendre(9, 13))
print(Bset(7, 34127))
print(Bfactors(23 * 23 * 59, Bset(7, 34127)))
print(least_res(27 + 33 * 16, 33))

1
[-1, 2, 7, 11, 17, 23, 29, 59]
[0, 0, 0, 0, 0, 2, 0, 1]
-6


Next, we build the table with values A and Bfactors(least_res(A^2 mod n)) for (limit) convergents.

In [235]:
def conv_fact_table(n, B, convs):
    table = []
    for A in convs:
        factored = Bfactors(least_res(A ** 2, n), B)
        if factored != 0:
            table.append([A, least_res(A ** 2, n), factored])
    return table

def split_table(table):  #  splits into two lists of A's and B factorisations
    a = []
    factors = []
    for i in range(len(table)):
        a.append(table[i][0])
        factors.append(table[i][2])
    return[a, factors]

testifeven takes a list of factorisations base B and checks if they are congruent mod 2. halfpower builds the number v with halved powers from the factorisations.

In [236]:
def testifeven(l, B):  #  l is list of lists, B is set of primes with -1
    for i in range(len(B)):
        sumcol = 0
        for j in range(len(l)):
            sumcol += l[j][i]
        if sumcol % 2 != 0:
            return 0
    return 1

def halfpower(l, B):  #  l is list of lists, B is set of primes with -1
    halvedpowers = 1
    for i in range(len(B)):
        sumcol = 0
        for j in range(len(l)):
            sumcol += l[j][i]
        halvedpowers *= B[i] ** int(sumcol/2)
    return halvedpowers

In [237]:
print(testifeven([[1, 2, 4, 0, 0, 18], [1, 2, 4, 0, 0, 13], [0, 0, 0, 2, 0, 3]], [-1, 2, 3, 5, 7, 11]))
print(halfpower([[0, 3, 2, 1], [0, 1, 2, 1]], [-1, 2, 3, 5]))  #  2^2 * 3^2 * 5^1

1
180


For a list of lists l and number d we get the list of lists which have positions corresponding to 1's in binary representation of d. We will use this to get partitions of factored A^2 mod n and check their dependence mod 2. <br />For a list A of convergents A_i, mult returns the product of the A_i on positions i corresponding to 1's in  binary representation of d.

In [238]:
def partitions(l, d):
    part = []
    d = f"{d:0{len(l)}b}"
    for i in range(len(d)):
        if d[i] == "1":
            part.append(l[i])
    return part

def mult(A, d):
    mult = 1
    d = f"{d:0{len(A)}b}"
    for i in range(len(d)):
        if d[i] == "1":
            mult = mult * A[i]
    return mult


In [239]:
print(partitions([[1, 3, 0, 0, 1], [1, 3, 1, 0, 0], [1, 6, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 3, 0, 1, 0]], 7))  # 000111
print(mult([11, 21, 2, 6, 3, 7], 11))  # 001011, 2 * 3 * 7

[[0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 3, 0, 1, 0]]
42


In [240]:
def sieve(convs, fac, B, n):
    for d in range(2 ** len(convs)):
        if len(partitions(fac, d)) >= 2 and testifeven(partitions(fac, d), B) == 1:
                u = mult(convs, d) % n
                v = halfpower(partitions(fac, d), B) % n
                if np.gcd(u - v, n) > 1:
                     return int(np.gcd(u - v, n))
    return 0

In [241]:
sieve([133, 401, 3877, 13369, 17246, 65107], [[1, 3, 0, 0, 1], [1, 3, 1, 0, 0], [1, 6, 0, 0, 0], [0, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 3, 0, 1, 0]], [-1, 2, 7, 11, 23], 17873)

293

Description of the algorithm: <br />Step 1: Start with n, limit, k. n is the number to be factored, limit and k are limits set by us to measure the depth of sieving. <br />Step 2: Build the factor base B, that is the set of first k prime numbers p for which n is a quadratic residue mod p. <br />Step 3: Calculate convergents of sqrt(n) A_i for i up to limit. We don't need values of B's for the sieve. <br />Step 4: For each A_i calculate the value A_i^2 mod n and keep the numbers A_i for which A_i^2 can be factored with primes in B, together with these factorisations as vectors of powers of the primes. <br />Step 5: Check sets of vectors form a linearly dependent set mod 2 and keep the corresponding A_i's together with such sets. <br />Step 6: Form u as the product of these A_i's and v as the product of primes which appear in the factor base with their powers halved. <br />Step 7: Calculate gcd(u - v, n) in an attempt to find a factor of n. If gcd(u - v, n) = 1, repeat from Step 5 with another linearly dependent set.

In [248]:
def cont_frac(n, limit, k):  # limit: how many steps/convergents to look for   k: number of primes in B set
    B = Bset(k, n)
    convs = conv_sqrt_n(n, limit)
    table = conv_fact_table(n, B, convs)
    convs = split_table(table)[0]  #  "good convergents"
    fac = split_table(table)[1]
    factor = sieve(convs, fac, B, n)
    if factor != 0:
        return f'a factor of {n} is {factor}'
    else:
        return "no factors found, try larger limit or k"
    

In [249]:
print(cont_frac(17873, 3, 3))
print(cont_frac(17873, 5, 5))
print(cont_frac(17873, 10, 4))
print(cont_frac(17873, 12, 5))



no factors found, try larger limit or k
no factors found, try larger limit or k
a factor of 17873 is 293
a factor of 17873 is 61
